In [1]:
from mb.gpt.models.connecter_type1 import BasicLLM
from mb.gpt.models.vlm_encoder_clip import VlmEncoderTest
from mb.gpt.models.text_encoder_smalvlm import TextEncoderTest

In [2]:
vlm_encoder = VlmEncoderTest(128,128)
text_encoder = TextEncoderTest(256,64)
model = BasicLLM(vlm_emb_dim=128, text_emb_dim=64, embedding_dim=256)
model

BasicLLM(embedding_dim=256)

In [3]:
for module in model.modules():
    print(module)

BasicLLM(embedding_dim=256)
Linear(in_features=192, out_features=256, bias=True)


In [4]:
import torch

data1 = torch.randn(10, 128)  # Simulated VLM embedding
data2 = torch.randn(10, 256)   # Simulated text embedding


In [5]:
data1[0,:10]

tensor([ 0.2105,  1.0456,  2.1830,  0.0887, -0.7237,  0.3205, -1.1705, -1.0274,
         0.2224,  0.1359])

In [6]:
vlm_data = vlm_encoder(data1)
text_data = text_encoder(data2)

In [7]:
vlm_encoder.state_dict()['linear.weight']

tensor([[ 0.0598, -0.0019,  0.0545,  ...,  0.0385,  0.0752,  0.0125],
        [-0.0160, -0.0144, -0.0794,  ...,  0.0388, -0.0173,  0.0681],
        [ 0.0487, -0.0142, -0.0423,  ..., -0.0376,  0.0085,  0.0322],
        ...,
        [ 0.0200,  0.0174, -0.0591,  ...,  0.0568, -0.0788, -0.0445],
        [ 0.0400,  0.0727, -0.0876,  ...,  0.0308, -0.0400,  0.0083],
        [ 0.0544, -0.0225, -0.0853,  ...,  0.0513,  0.0734, -0.0573]])

In [8]:
optimizer = torch.optim.Adam(
    list(vlm_encoder.parameters()) + list(text_encoder.parameters()) + list(model.parameters()),
    lr=1e-3
)
optimizer.zero_grad()
output = model(vlm_data, text_data,None)
loss = output.sum()  # Dummy loss
loss.backward()
optimizer.step()

In [9]:
vlm_encoder.state_dict()['linear.weight']

tensor([[ 0.0608, -0.0029,  0.0555,  ...,  0.0395,  0.0762,  0.0135],
        [-0.0170, -0.0134, -0.0804,  ...,  0.0378, -0.0183,  0.0671],
        [ 0.0497, -0.0152, -0.0413,  ..., -0.0366,  0.0095,  0.0332],
        ...,
        [ 0.0210,  0.0164, -0.0581,  ...,  0.0578, -0.0778, -0.0435],
        [ 0.0410,  0.0717, -0.0866,  ...,  0.0318, -0.0390,  0.0093],
        [ 0.0534, -0.0215, -0.0863,  ...,  0.0503,  0.0724, -0.0583]])